# Lamieux Method For Brain Image Segmentation

In [1]:
from __future__ import print_function

import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, FloatSlider

import SimpleITK as sitk
import numpy as np

from myshow import myshow, myshow3d

Loading data and initial visualization...

In [2]:
img_T1 = sitk.ReadImage("../data/IXI002-Guys-0828-MPRAGESEN_-s256_-0301-00003-000001-01.nii")

# To visualize the labels image in RGB with needs a image with 0-255 range
img_T1_255 = sitk.Cast(sitk.RescaleIntensity(img_T1), sitk.sitkUInt8)

myshow3d(img_T1)

interactive(children=(IntSlider(value=74, description='z', max=149), Output()), _dom_classes=('widget-interact…

### Step 1: Calculating the Foreground Threshold Level

In [9]:
def OstsuThresh(img):
    otsuFilter = sitk.OtsuThresholdImageFilter()
    otsuFilter.SetOutsideValue(255)
    otsuFilter.SetInsideValue(0)
    mask = otsuFilter.Execute(img)
    return mask, otsuFilter.GetThreshold()

In [14]:
mask, threshOtsuValue = OstsuThresh(img_T1)
I_scf = threshOtsuValue / 3.5 + 6
# To visualize the labels image in RGB with needs a image with 0-255 range
myshow(sitk.LabelOverlay(img_T1_255, mask), "Otsu Thresholding")
print('Threshold Otshu value: ', threshOtsuValue)
print('Estimated cerebrospinal fluid (CSF) intensity : ', I_scf)

interactive(children=(IntSlider(value=74, description='z', max=149), Output()), _dom_classes=('widget-interact…

Threshold Otshu value:  179.0
Estimated cerebrospinal fluid (CSF) intensity :  57.142857142857146


## Step 2 : Initial Brain Disconnection 

In [26]:
def Threshold(img, threshValue):
    thresh = sitk.BinaryThresholdImageFilter()
    # defines threshold window
    thresh.SetUpperThreshold(threshValue);
    thresh.SetLowerThreshold(0);
    # defines inner and outer values
    thresh.SetOutsideValue(255);
    thresh.SetInsideValue(0);
    # apply threshold
    return thresh.Execute(img);

def InitialBrainDisconnection(img, threshOtsuValue, threshMult):
    # Compute a mask based on ostu value
    threshValue = threshMult * threshOtsuValue / 3
    mask = Threshold(img, threshValue)
    # Connectivity Components Analysis
    components = sitk.ConnectedComponent(img, mask, False)
    # reorder componends
    reorderedComponents = sitk.RelabelComponent(components)
    return reorderedComponents

In [28]:
dst = InitialBrainDisconnection(img_T1, threshOtsuValue, 6)
myshow(sitk.LabelOverlay(img_T1_255, dst), "Connected Component Labels")

interactive(children=(IntSlider(value=74, description='z', max=149), Output()), _dom_classes=('widget-interact…